In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

import seaborn as sns
sns.set_context('notebook')
sns.set(style="whitegrid", font_scale=1.5)
sns.despine()
sns.set_color_codes()

import matplotlib.pyplot as plt
plt.xlim(0, 1)
plt.ylim(0, None)
from matplotlib.ticker import MaxNLocator
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = [20, 5]

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import logging
logging.basicConfig(level=logging.INFO, stream=sys.stdout)

import vtk
import numpy as np
import nibabel as nib
from six import iteritems
from scipy.stats import norm
from nibabel.affines import apply_affine
from vtk.util import numpy_support as ns
from dipy.tracking.streamline import values_from_volume

In [2]:
main_folder = '/Users/imag2/Dropbox/IMAG2/MAR/'

In [3]:
def load_nii(fname):
    img = nib.load(fname)
    return img.get_fdata(), img.affine

def read_vtk(filename):
    if filename.endswith('xml') or filename.endswith('vtp'):
        polydata_reader = vtk.vtkXMLPolyDataReader()
    else:
        polydata_reader = vtk.vtkPolyDataReader()

    polydata_reader.SetFileName(filename)
    polydata_reader.Update()

    polydata = polydata_reader.GetOutput()

    return vtkpolydata_to_tracts(polydata)

def vtkpolydata_to_tracts(polydata):
    result = {'lines': ns.vtk_to_numpy(polydata.GetLines().GetData()),
              'points': ns.vtk_to_numpy(polydata.GetPoints().GetData()),
              'numberOfLines': polydata.GetNumberOfLines()
             }

    data = {}
    if polydata.GetPointData().GetScalars():
        data['ActiveScalars'] = polydata.GetPointData().GetScalars().GetName()
    if polydata.GetPointData().GetVectors():
        data['ActiveVectors'] = polydata.GetPointData().GetVectors().GetName()
    if polydata.GetPointData().GetTensors():
        data['ActiveTensors'] = polydata.GetPointData().GetTensors().GetName()

    for i in range(polydata.GetPointData().GetNumberOfArrays()):
        array = polydata.GetPointData().GetArray(i)
        np_array = ns.vtk_to_numpy(array)
        if np_array.ndim == 1:
            np_array = np_array.reshape(len(np_array), 1)
        data[polydata.GetPointData().GetArrayName(i)] = np_array

    result['pointData'] = data

    tracts, data = vtkpolydata_dictionary_to_tracts_and_data(result)
    return tracts, data

def vtkpolydata_dictionary_to_tracts_and_data(dictionary):
    dictionary_keys = {'lines', 'points', 'numberOfLines'}
    if not dictionary_keys.issubset(dictionary):
        raise ValueError("Dictionary must have the keys lines and points" + repr(dictionary))

    tract_data = {}
    tracts = []

    lines = np.asarray(dictionary['lines']).squeeze()
    points = dictionary['points']

    actual_line_index = 0
    number_of_tracts = dictionary['numberOfLines']
    original_lines = []
    for _ in range(number_of_tracts):
        tracts.append(points[lines[actual_line_index + 1:actual_line_index + lines[actual_line_index] + 1]])
        original_lines.append(
            np.array(lines[actual_line_index + 1:actual_line_index + lines[actual_line_index] + 1], copy=True))
        actual_line_index += lines[actual_line_index] + 1

    if 'pointData' in dictionary:
        point_data_keys = [it[0] for it in iteritems(dictionary['pointData']) if isinstance(it[1], np.ndarray)]

        for k in point_data_keys:
            array_data = dictionary['pointData'][k]
            if k not in tract_data:
                tract_data[k] = [array_data[f] for f in original_lines]
            else:
                np.vstack(tract_data[k])
                tract_data[k].extend([array_data[f] for f in original_lines[-number_of_tracts:]])

    return tracts, tract_data

In [4]:
def get_hist(bundle, metric, affine):
    def streamlines_mapvolume(streamlines, volume, affine):
        mapping = values_from_volume(volume, streamlines, affine)
        return mapping

    scalar_measurement = streamlines_mapvolume(bundle, metric, affine)
    scalar_measurement = [item for sublist in scalar_measurement for item in sublist]
    return np.asarray(scalar_measurement)

def get_length(bundle):
    def _get_length(tract):
        return ((((tract[1:] - tract[:-1]) ** 2).sum(1)) ** .5).sum()
    tracts_lengths = [_get_length(s) for s in bundle]
    return tracts_lengths

In [5]:
def save_distr(l5, s1, s2, s3, s4, FA, affine):
    l5_series = pd.Series(get_hist(l5, FA, affine), name="L5")
    s1_series = pd.Series(get_hist(s1, FA, affine), name="S1")
    s2_series = pd.Series(get_hist(s2, FA, affine), name="S2")
    s3_series = pd.Series(get_hist(s3, FA, affine), name="S3")
    s4_series = pd.Series(get_hist(s4, FA, affine), name="S4")
    
    frame = pd.DataFrame(
        {'L5': l5_series,
         'S1': s1_series,
         'S2': s2_series,
         'S3': s3_series,
         'S4': s4_series
        } 
    )
  
    frame.to_csv('/Users/imag2/Desktop/FA_1-4-02.csv')

In [6]:
'''
for file in os.listdir(main_folder):
    current_path = os.path.join(main_folder, file)
    if '.vtk' in current_path.lower() and 'l5' in current_path.lower():
        l5 = read_vtk(current_path)[0]
    elif '.vtk' in current_path.lower() and 's1' in current_path.lower():
        s1 = read_vtk(current_path)[0]
    elif '.vtk' in current_path.lower() and 's2' in current_path.lower():
        s2 = read_vtk(current_path)[0]
    elif '.vtk' in current_path.lower() and 's3' in current_path.lower():
        s3 = read_vtk(current_path)[0]
    elif '.vtk' in current_path.lower() and 's4' in current_path.lower():
        s4 = read_vtk(current_path)[0]
    elif '.nii' in current_path.lower() and 'fa' in current_path.lower():
        FA, affine = load_nii(current_path)
        FA = (FA - np.amin(FA)) / (np.amax(FA) - np.amin(FA))

save_distr(l5, s1, s2, s3, s4, FA, affine)
'''

"\nfor file in os.listdir(main_folder):\n    current_path = os.path.join(main_folder, file)\n    if '.vtk' in current_path.lower() and 'l5' in current_path.lower():\n        l5 = read_vtk(current_path)[0]\n    elif '.vtk' in current_path.lower() and 's1' in current_path.lower():\n        s1 = read_vtk(current_path)[0]\n    elif '.vtk' in current_path.lower() and 's2' in current_path.lower():\n        s2 = read_vtk(current_path)[0]\n    elif '.vtk' in current_path.lower() and 's3' in current_path.lower():\n        s3 = read_vtk(current_path)[0]\n    elif '.vtk' in current_path.lower() and 's4' in current_path.lower():\n        s4 = read_vtk(current_path)[0]\n    elif '.nii' in current_path.lower() and 'fa' in current_path.lower():\n        FA, affine = load_nii(current_path)\n        FA = (FA - np.amin(FA)) / (np.amax(FA) - np.amin(FA))\n\nsave_distr(l5, s1, s2, s3, s4, FA, affine)\n"

In [27]:
df = pd.DataFrame()

for dirs in os.listdir(main_folder):
    name = dirs
    current_dir = os.path.join(main_folder,dirs)
    if os.path.isdir(current_dir):
        for file in os.listdir(current_dir):
            current_path = os.path.abspath(os.path.join(main_folder, current_dir, file))
            if '.vtk' in current_path.lower() and 's4' in current_path.lower():
                l5 = pd.Series(get_length(read_vtk(current_path)[0]))
                #print(l5)
                df[name] = l5

print(df.to_csv('/Users/imag2/Desktop/S4.csv'))

/Users/imag2/miniconda3/envs/IMAG2_Utilities/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.


None
